<a href="https://colab.research.google.com/github/Lau-Jr/data-engineering-dump/blob/main/dataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 27 kB/s 
     |████████████████████████████████| 198 kB 45.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=32682d6dd2935a0abb16a6137339c8a2f618617315617c6ce84cad643960f851
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [3]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [4]:
sc = SparkContext()
spark = SparkSession(sc)


In [5]:

data = [
        ('laurent',25,'asha'),
        ('diana',21,'asha'),
        ('yusra',16,'asha'),
        ('france',15,'ngara'),
        ('zion',12,'upendo'),
        ('pu',10,'upendo'),
        ('prince',7,'upendo'),
        ('gato',3,'upendo'),
        ('edo',1,'upendo'),
        ]
names = ['name','age','mother']
dataRDD = sc.parallelize(data)

In [6]:
watotoDF = dataRDD.toDF()
new_watotoDF = spark.createDataFrame(data,schema = names)

In [7]:
house_df = spark.read.csv('/content/sample_data/california_housing_train.csv',header = True,inferSchema =True)
house_df.show(2)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
only showing top 2 rows



In [8]:
ansombe_js = spark.read.json('/content/sample_data/anscombe.json')
ansombe_js.show(2)

+------+----+----+---------------+
|Series|   X|   Y|_corrupt_record|
+------+----+----+---------------+
|  null|null|null|              [|
|     I|10.0|8.04|           null|
+------+----+----+---------------+
only showing top 2 rows



In [9]:
house_df.select(['total_rooms','total_bedrooms']).show(2)

+-----------+--------------+
|total_rooms|total_bedrooms|
+-----------+--------------+
|     5612.0|        1283.0|
|     7650.0|        1901.0|
+-----------+--------------+
only showing top 2 rows



In [10]:
house_df.filter(house_df.total_rooms > 5000).show(3)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|
|  -116.06|   34.15|              15.0|    10377.0|        2331.0|    4507.0|    1807.0|       2.2466|           66800.0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
only showing top 3 rows



In [11]:
house_df.groupBy('latitude').count().orderBy('count',ascending = False).count()

840

In [12]:
house_df.groupBy('latitude').count().dropDuplicates().orderBy('count',ascending = False).count()

840

In [14]:
watotoDF = watotoDF.withColumnRenamed("_3",'mother')

In [16]:
watotoDF.columns

['_1', '_2', 'mother']

In [17]:
watotoDF.createOrReplaceTempView('kids')

In [18]:
spark.sql('select * from kids where mother = "asha" ').show()

+-------+---+------+
|     _1| _2|mother|
+-------+---+------+
|laurent| 25|  asha|
|  diana| 21|  asha|
|  yusra| 16|  asha|
+-------+---+------+



In [29]:
watotoDF.collect()[0].asDict()['mother']

'asha'

In [33]:
watotoDF.groupBy('Mother').count().orderBy('count',ascending = False).show()

+------+-----+
|Mother|count|
+------+-----+
|upendo|    5|
|  asha|    3|
| ngara|    1|
+------+-----+

